### Import Libraries and Initialisation

In [87]:
import requests
from base64 import b64encode
import json
import pandas as pd
from datetime import datetime, date, timedelta

import sys
import yaml as yml
import pyodbc as cnn

import re
from dateutil.relativedelta import relativedelta
from tqdm import tqdm, trange
   
### THis is to update accordingly    
apikey = "fz/ocDZn9rHm9ZAupGsDNjoBZGSnHO+8vmU/xWiPPvvFzPE0STJnTbnvu6/FvyOpUhp/TBu0txZ4xZYz6J8iyxkUDkBowVTiDflAvH0qdi5NzWSW+iq8L7j77/Qg77/6Wkj1JvxGkBTOtFOpV9saGw=="
clientId = "33604187fc197c87eb8d2f553935ef8e"

page = 1
pagesize = 1000

# Encode the credentials with base64
username = apikey
password = ''
encoded_credentials = b64encode(bytes(f'{username}:{password}',encoding='ascii')).decode('ascii')

# Use Basic Authorization
auth_header = f'Basic {encoded_credentials}'
print(f'Auth header: {auth_header}')

# Open the config file
with open(r'../dbconfig.yaml') as file:
    config = yml.full_load(file)

Auth header: Basic Znovb2NEWm45ckhtOVpBdXBHc0ROam9CWkdTbkhPKzh2bVUveFdpUFB2dkZ6UEUwU1RKblRibnZ1Ni9GdnlPcFVocC9UQnUwdHhaNHhaWXo2SjhpeXhrVURrQm93VlRpRGZsQXZIMHFkaTVOeldTVytpcThMN2o3Ny9RZzc3LzZXa2oxSnZ4R2tCVE90Rk9wVjlzYUd3PT06


### Establish Database Connection

In [88]:
# Establish the connection with database server
conn = cnn.connect('DRIVER='+config['instances']['driver']+
                   ';SERVER='+config['instances']['servername']+
                   ';DATABASE='+config['instances']['database']+
                   ';UID='+config['instances']['username']+
                   ';PWD='+config['instances']['password'])

conn.autocommit = False

In [89]:
conn

###  Initial Objects Declaration

In [90]:
campaign_email_action_list = ["opens", "clicks", "unsubscribes", "bounces", "spam"]

# Base API parameters
params = {"page": page, "pagesize": pagesize, "orderfield": "email", "orderdirection": "asc" }

## The list of columns to select from the original API response. Only these columns will be stored into the Database
desired_columns = ["campaignId", "EmailAddress", "Date"]

## Place holder to store the downloaded data
final_campaign_action_data = {"opens": {"data": pd.DataFrame(), "destination_table_name": "campaign_email_opens"},
                              "clicks":{"data": pd.DataFrame(), "destination_table_name": "campaign_email_clicks"},
                              "unsubscribes": {"data": pd.DataFrame(), "destination_table_name": "campaign_email_unsubscribes"},
                              "bounces": {"data": pd.DataFrame(), "destination_table_name": "campaign_email_bounces"},
                              "spam": {"data": pd.DataFrame(), "destination_table_name": "campaign_email_spam"} }

####### Initialise the config related variable ######
# start time of processing
start_proc_time = datetime.now()

# Data loading method - "full" for the first time and "incremental" since second run
# Check loading configuration #METHOD: full/incremental
cfg_load_method = config['loading']['method']

#START_DATE_FULL_LOAD: starting date of data to be loaded (if method is full-load)
cfg_start_date = config['loading']['start_date_full_load']

#DAY_UPDATE_LAG: number of days of data lag
cfg_update_lag = config['loading']['day_update_lag']

# Calculate end date (the last latest day to extract/download data according to config 'cfg_update_lag')
cfg_end_date = datetime.now() - relativedelta(days = cfg_update_lag)

# number of days to consider a campaign is active
cfg_cmp_active_period_length = config['loading']['campaign_active_period_length']

# number of months to consider a campaign is active
default_full_load_month = config['loading']['default_full_load_month']

In [93]:
print("Begining of historical data to be stored:", cfg_start_date)
print("Length of duration Campaign being monitored:",cfg_cmp_active_period_length, "days")

print("Default full_load month:", default_full_load_month)
print("Method full (or) incremental ?", cfg_load_method)

Begining of historical data to be stored: 2010-01-01
Length of duration Campaign being monitored: 30 days
Default full_load month: 2
Method full (or) incremental ? full


### Common Functions


In [95]:
### Get the sent campaign
def getAllSentCampaigns(clientId):
    campaign_detail_url = f'https://api.createsend.com/api/v3.2/clients/{clientId}/campaigns.json'
    response = requests.get(campaign_detail_url, headers={'Authorization': auth_header } ) 
    response_campaign_list = response.json()
    df_campaign_list = pd.DataFrame.from_dict(response_campaign_list)
    return df_campaign_list

def getFilteredActiveCampaign(df_campaign_list): 
    '''
    The function filters the campaigns which are still within n(for instance: 120) days since the email sent date.
    Basically, get all the still-active campaigns under monitoring period set by config
    '''
    # Get n() days ago from now
    last_active_date = get_datetime_Ndays_ago(cfg_cmp_active_period_length)
    
    # The campaign with SentDate which are greater than xxx, means within the monitoring period 
    # Get the index of campaigns running for less than n days 
    all_active_campaign = df_campaign_list['SentDate'] > last_active_date 
    
    # Filter out only the active campaigns
    campaigns_on_active_period = df_campaign_list.loc[all_active_campaign]
    
    return campaigns_on_active_period

def get_datetime_Ndays_ago(n):
    '''
    returns: string format of datatime - "%Y-%m-%d %H:%M:%S"
    ''' 
    N_days_ago_datetime = datetime.now() - timedelta(days=n)
    return N_days_ago_datetime.strftime("%Y-%m-%d %H:%M:%S")
       
def get_next_NDays(dateFrom, n):
    '''
    returns: string format of datatime - "%Y-%m-%d %H:%M:%S"
    arguments:
    ----------
    dateFrom - string type, the date to begin count N number of days from
    n - integer, numbers of days to be added into the dateFrom
    
    ''' 
   
    ## Convert to datetime type to do date addition
    begindate = datetime.strptime(dateFrom, "%Y-%m-%d %H:%M:%S") 
    
    next_N_days_datetime = begindate + timedelta(days=n)
    ## return the string format of next N date
    return next_N_days_datetime.strftime("%Y-%m-%d %H:%M:%S")

def cal_number_of_days_to_campaingnEndDay(dateFrom, n):
    '''
    This function helps to get the count of days between two dates.
    The intention is to get how many of day count for data to be downloaded from the start date given in the argument.
    The campaign end date varies and it could be either in the past or future.
    It has to be calculated based on current time.
    
    Arguments:
    ----------
    dateFrom: datetime
            the start date to download data
    n: integer
       The number of days, it depends on the config in which refers to "campaign_active_period_length"
       
    Result:
    ------
    number_of_days_required: integer
                            The numbers of days between the "dateFrom" and calculated "end_day"
    
    '''
     
    # Get the campaign end date    
    datetime_object = datetime.strptime(dateFrom, '%Y-%m-%d %H:%M:%S')
    next_N_days_datetime = datetime_object + timedelta(days=n)
    
    # If the campaign end date is already passed
    if (datetime.now() > next_N_days_datetime):
        end_day = next_N_days_datetime
        
    # If the campaign end date is in future, it should be n(2) days ago
    elif ( datetime.now() < next_N_days_datetime):
        end_day = get_datetime_Ndays_ago(cfg_update_lag)
        end_day = datetime.strptime(end_day, '%Y-%m-%d %H:%M:%S')

    # Calculate the difference between two dates
    number_of_days_required = end_day - datetime_object 
    return(number_of_days_required.days)


In [96]:
def get_last_success_load_time(table):
    '''
    This function queries the last success log of data insert for a particular table
    Argument:
    --------
    table: string
          It is the name of table such as
          "campaign_email_opens", "campaign_email_clicks", "campaign_email_bounces", "campaign_email_unsubscribes"
    '''
    
    ### Check the names of tables here TODO
    load_log = pd.read_sql(  '  SELECT TOP 1'
                             '         load_date, '
                             '         method, '
                             '         data_start_date, ' 
                             '         data_end_date '
                             '  FROM log_success_load '
                             '  WHERE  table_name = \''+table+'\' '
                             '  ORDER BY end_runtime DESC ',
                          conn)

    #if no data is available in the table (log_success_load), then no data has been load before
    #so, applying full load instead (4 months of data)    
    if (load_log.shape[0] == 0):
        cfg_start_date = datetime.now() - relativedelta(months = default_full_load_month)
        cfg_load_method = 'full [def]'
        print('#No data is available in the database. Applying \'Full Load\' From \''+cfg_start_date.strftime('%Y-%m-%d')+'\'')
    else:
        print("load_log.iloc[0]['data_end_date']", load_log.iloc[0]['data_end_date'])
        cfg_start_date = load_log.iloc[0]['data_end_date'] + relativedelta(days = 1)
        print(cfg_start_date)
        print('#Applying \'Incremental Load\' From \''+load_log.iloc[0]['data_end_date'].strftime("%Y-%m-%d %H:%M:%S")+'\'')
    return cfg_start_date

def set_query_start_date(tableName):
    ## This function returns the string format of datetime object
    
    ## Retrive the last success load date from DB of the respective table
    query_startDate = get_last_success_load_time(tableName)
    ### convert to string to return
    query_startDate = query_startDate.strftime("%Y-%m-%d %H:%M:%S")
    
    return query_startDate

def insert_into_db(row, cursor, table):
     cursor.execute(f'INSERT INTO {table}'
               '  (campaign_id, email_address, action_date ) VALUES   (?,?,?) ',
               row['campaignId'], row['EmailAddress'], row['Date'])


### Get the Sent Campaign List

In [97]:
###### Initially, get the list all the past Campaigns #######
df_all_sent_campaigns_list = getAllSentCampaigns(clientId)

### Data Acquistion through Campaign Monitor API

In [98]:
###### Identify the requirement of load method ######
# if Full, all the campaigns in the history will be downloaded
# if Incremental, only active campaigns will be downloaded
if (cfg_load_method == 'full'):
    campaigns_on_active_period = df_all_sent_campaigns_list
elif (cfg_load_method == 'incremental'):
    campaigns_on_active_period = getFilteredActiveCampaign(df_all_sent_campaigns_list)

    
###### Loop by each campaign
for i in range(len(campaigns_on_active_period)) : 
    
    # Get the identifier of the campaign looping now
    campaign_startDate = campaigns_on_active_period.loc[i, 'SentDate']
    campaign_id = campaigns_on_active_period.loc[i, 'CampaignID'] 
    
    
    # Count the number of days between C-SentDate and End-Date
    number_of_days = cfg_cmp_active_period_length 
    
    ###### Loop by each action
    for action in campaign_email_action_list:
        
        ### Set the Query Start Date
        query_startDate = campaign_startDate
        if(cfg_load_method == 'incremental'):
            
            ## Set the load starting time according to the last loaded time recorded in the db-log
            query_startDate = set_query_start_date(final_campaign_action_data[action]['destination_table_name'])
            
            ## Overwriting number_of_days - if "incremental" to get the exact required count of loop
            number_of_days = cal_number_of_days_to_campaingnEndDay(campaign_startDate, cfg_cmp_active_period_length)
       
        ## Define the API End point  
        url_campaign_opens = f"https://api.createsend.com/api/v3.2/campaigns/{campaign_id}/{action}.json"
        # placeholder for each action's data
        df_campaign_email_action = pd.DataFrame()
        all_records = []
        
        #### Loop by each day for one action for one campaign
        for x in range(number_of_days):
             # Increase the next date count in every round of loop
             print("query_startDate", query_startDate)
            
             queryDate = get_next_NDays(query_startDate, x)
             # Update the API param "date"
             params["date"] = queryDate
            
             # active the GET request
             response = requests.get(url_campaign_opens, headers={'Authorization': auth_header }, params = params) 
             response_campaign_action = response.json()
             
             # If data is empty , skip the loop  
             if(len(response_campaign_action["Results"]) == 0): continue
             # Convert API response in dict format to Dataframe format
             response_dataframe = pd.DataFrame.from_dict(response_campaign_action["Results"])
               
             # Add "campaignId" value to the column
             response_dataframe["campaignId"] = campaign_id
             # Select the desired columns only from the entire response to save into the DB
             response_dataframe = response_dataframe[desired_columns]
             # Add up the dataframe to the result placeholder
             df_campaign_email_action = df_campaign_email_action.append(response_dataframe, ignore_index=True, sort=False) 
        
        if(len(df_campaign_email_action.index) > 0):           
            campaign_email_action_dataset = df_campaign_email_action.drop_duplicates()
            final_campaign_action_data[action]["data"] = final_campaign_action_data[action]["data"].append(campaign_email_action_dataset, ignore_index=True, sort=False) 
            
end_proc_time = datetime.now()

query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 21:00:00
query_startDate 2021-06-28 2

query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 15:15:00
query_startDate 2021-06-28 1

query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 16:30:00
query_startDate 2020-12-08 1

query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 14:20:00
query_startDate 2020-12-07 1

In [86]:
# final_campaign_action_data

### Insert Into Database


In [99]:
#### Insert data by action (table by table)
for action in campaign_email_action_list:
        
        cursor = conn.cursor()
        # Name of table to store data
        table_name = final_campaign_action_data[action]["destination_table_name"] 
        campaign_data = final_campaign_action_data[action]["data"]
        if (campaign_data.empty) : continue
        try:
            #if full-load, delete all the records from the table, log records from the log table
            if (cfg_load_method == 'full'):
                cursor.execute(f'DELETE FROM {table_name}')
                cursor.execute(f'DELETE FROM log_success_load WHERE table_name = \''+table_name+'\' ')
            
            #insert dataframe to DB table
            campaign_data.apply(insert_into_db, cursor = cursor, axis = 1, table= table_name)
            
            cursor.execute('INSERT INTO '
                   '       log_success_load '
                   '       (load_date, method, data_start_date, data_end_date, start_runtime, end_runtime, table_name) '
                   'VALUES (?,?,?,?,?,?,?)',
                  start_proc_time.strftime('%Y-%m-%d'),
                  cfg_load_method,
                  cfg_start_date.strftime('%Y-%m-%d'),
                  cfg_end_date.strftime('%Y-%m-%d'),
                  start_proc_time,
                  end_proc_time, 
                  table_name) 

            conn.commit()
        except:
            conn.rollback()
            print(sys.exc_info())
        finally:  
            cursor.close()
            
####### Close the connection #########
conn.close()